# Back Reaction

## Initialization

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

# np.set_printoptions(threshold=np.nan)

## Load Reference Results

These reference results start with a delta-function like injection at a redshift of $(1+z) = 31$ of 2 photons at the energy given by the file name. The high energy photons are stored in `file_photonspectrum`, low energy photons in `file_lowengphot` and low energy electrons in `file_lowengelec`. The photon energy abscissa is `file_eng` (both high and low) while the electron energy abscissa is `file_eleceng`. 

In [3]:
eng_ind = 299

# Check that this matches with the input 

file_name = {99: "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tf_z_3.100E+01_nstep_2049_logE_5.328E+00_xe_1.000E-04.fits",
            199: "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tf_z_3.100E+01_nstep_2049_logE_7.183E+00_xe_1.000E-04.fits",
            298: "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tf_z_3.100E+01_nstep_2049_logE_9.020E+00_xe_1.000E-04.fits",
            299: "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tf_z_3.100E+01_nstep_2049_logE_9.038E+00_xe_1.000E-04.fits", 
            399: "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tf_z_3.100E+01_nstep_2049_logE_1.089E+01_xe_1.000E-04.fits",
            499: "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tf_z_3.100E+01_nstep_2049_logE_1.275E+01_xe_1.000E-04.fits",
            2993: "/Users/hongwan/Dropbox (MIT)/Photon Deposition/tf_z_3.100E+01_nstep_18_logE_9.038E+00_xe_1.000E-04.fits"}

file = fits.open(file_name[eng_ind])

file_eng     = file[1].data['energy'][0,:]
file_eleceng = file[1].data['eleceng'][0,:]
file_rs      = file[1].data['redshift'][0,:]

file_photonspectrum = file[1].data['photonspectrum'][0,:,:]
file_lowengphot     = file[1].data['lowengphot'][0,:,:]
file_lowengelec     = file[1].data['lowengelec'][0,:,:]
file_cmbloss        = file[1].data['cmblosstable'][0,:]

Turn the data into `Spectra` objects. 

In [4]:
file_highengphot_spectra = Spectra([Spectrum(file_eng, photspec, rs=rs) 
    for photspec,rs in zip(file_photonspectrum, file_rs)])

file_lowengphot_spectra = Spectra([Spectrum(file_eng, photspec, rs=rs) 
    for photspec,rs in zip(file_lowengphot, file_rs)])

file_lowengelec_spectra = Spectra([Spectrum(file_eleceng, elecspec, rs=rs) 
    for elecspec,rs in zip(file_lowengelec, file_rs)])

## Photon Cooling

Now we cool the photons that result from the high energy photon injection. Photons below 3.1 keV are considered low energy photons. For high energy photons, greater than 3.1 keV, we have transfer functions into high energy photons, low energy photons and low energy electrons. Let's load them first. These are saved as arrays of `TransFuncList` objects, for a list of 10 different levels of $x_e$.

In [5]:
user = 'hongwan'

highengphot_tflist_arr_raw = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_list.raw", "rb"))
lowengphot_tflist_arr_raw  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_list.raw", "rb"))
lowengelec_tflist_arr_raw  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_list.raw", "rb"))

For checks, we may want to use just one particular set of transfer functions. If so, we should run this cell.

In [6]:
tflist_xe_ind = 0

print(0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4]))

highengphot_tflist_arr = [highengphot_tflist_arr_raw[tflist_xe_ind]]
lowengphot_tflist_arr  = [lowengphot_tflist_arr_raw[tflist_xe_ind]]
lowengelec_tflist_arr  = [lowengelec_tflist_arr_raw[tflist_xe_ind]]

[4.53978687e-05 2.74578156e-04 1.65880108e-03 9.95180187e-03
 5.73241759e-02 2.68941421e-01 6.89974481e-01 9.30861580e-01
 9.87871565e-01 9.97974680e-01 9.99664650e-01]


In [29]:
np.set_printoptions(threshold=np.nan)

test = highengphot_tflist_arr[0][1]
print(test.in_eng[330])
print(test.in_eng[331])
interp_test = test.at_val([1.06e9], test.eng)
test_29=highengphot_tflist_arr[0][1]
utils.compare_arr([test[330].eng,test[330].N, interp_test[0].N, test[331].N])

1042988825.9459214
1097049296.408165
[[1.03931093e-004 3.79154528e-003 3.79956889e-003 3.81660718e-003]
 [1.12262959e-004 4.35177404e-003 4.36122581e-003 4.38129680e-003]
 [1.21262768e-004 4.99117531e-003 5.00229955e-003 5.02592206e-003]
 [1.30984067e-004 6.94607409e-003 6.94973181e-003 6.95749905e-003]
 [1.41484694e-004 6.94027482e-003 6.96486783e-003 7.01709150e-003]
 [1.52827128e-004 7.95069352e-003 7.96926113e-003 8.00868976e-003]
 [1.65078853e-004 1.00763523e-002 1.00650809e-002 1.00411458e-002]
 [1.78312765e-004 1.18842516e-002 1.19399745e-002 1.20583030e-002]
 [1.92607603e-004 1.26506004e-002 1.26814276e-002 1.27468897e-002]
 [2.08048418e-004 1.44996317e-002 1.45359496e-002 1.46130713e-002]
 [2.24727080e-004 2.01017520e-002 2.00576036e-002 1.99638538e-002]
 [2.42742824e-004 2.01758638e-002 2.03039782e-002 2.05760313e-002]
 [2.62202840e-004 2.30346825e-002 2.30947174e-002 2.32222026e-002]
 [2.83222911e-004 2.90653587e-002 2.89420798e-002 2.86802950e-002]
 [3.05928103e-004 3.44359

0

In [ ]:
%autoreload

test = highengphot_tflist_arr[0][0]
test_29=highengphot_tflist_arr[0][1]

plt.figure()
ax = plt.gca()

# test.plot(ax, ind=np.array([200,201]), indtype='ind')
# test.plot(ax, ind=np.array([1.471773e6, 1.495e6, 1.527932e6]), indtype='in_eng')
# test.plot(ax, ind=10**np.array([9.038]), indtype='in_eng')
test.plot(ax, ind=[331])
test_29.plot(ax, ind=[331])
# test.plot(ax, ind=np.array([10**7.4]), indtype='in_eng')
file_highengphot_spectra.switch_spec_type()
# file_highengphot_spectra.plot(ax, ind=1)
file_highengphot_spectra.switch_spec_type()
ax.set_xscale('log')
ax.set_yscale('log')
plt.axis([1e-4, 2e10, 1e-12, 1e3])

In [ ]:
a = test.at_val([9.3e6], test.eng)
print(test.in_eng[331])
print(test.in_eng[301])
print(10**9.038)
print(test[300].toteng())
print(test[301].toteng())
print(a[0].toteng())

## Inputs

Next, we define the input spectrum. Currently, `eleceng` and `photeng` must be defined as follows for accuracy, although `n` can be adjusted for speed. Here, we define a delta-function like injection at `mwimp_arr[eng_ind]`, which will correspond exactly with the reference file chosen. Note that `spectools.rebin_N_arr` produces a `Spectrum` object of type `'dNdE'` by default, so `Spectrum.switch_spec_type()` is required, since all of the transfer functions evolve `Spectrum` objects of type `'N'`. 

In [ ]:
# mwimp_arr = np.array(highengphot_tflist_arr[0].in_eng)

mwimp_arr = {99: 10**5.328, 199: 10**7.183, 298: 10**9.020, 299: 10**9.038, 
            399: 10**10.89, 499: 10**12.75}

n = 2

eleceng = lowengelec_tflist_arr[0].eng
photeng = highengphot_tflist_arr[0].eng

step = np.log(photeng[1]) - np.log(photeng[0])
test = np.exp(np.arange(np.log(photeng[0]), np.log(9.9e-5), -step))
photeng = np.append(np.flipud(test[1:]), photeng)

# eleceng = eleceng[1::n]
# photeng = photeng[1::n]

phot_spec_init = spectools.rebin_N_arr(np.array([2]), 
                                      np.array([mwimp_arr[eng_ind]]),
                                      photeng)
phot_spec_init.switch_spec_type()

To evolve the spectrum and allow for easy processing later, the high energy photon transfer function must be square. We make the abscissa for the transfer function `photeng` by `photeng` (injected energy by output energy), and simply pad the additional rows needed with zeros. For convenience, we also use `photeng` by `photeng` for the low energy photon transfer function, and `photeng` by `eleceng` for the low energy electron transfer function. The definitions for high and low energies are listed below. 

In [ ]:
#Split photeng into high and low energy. 
photeng_high = photeng[photeng >= 60]
photeng_low  = photeng[photeng < 60]

# Split eleceng into high and low energy. 
eleceng_high = eleceng[eleceng > 3500]
eleceng_low  = eleceng[eleceng <= 3500]


for highengphot_tflist in highengphot_tflist_arr:
    
    highengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)

    for tf in highengphot_tflist:
        # tf.rebin(photeng_high)
        tf.rebin(photeng)
        
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size, 0), (0, 0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    highengphot_tflist.eng = photeng
    highengphot_tflist.in_eng = photeng
    
# lowengphot_tflist.in_eng set to photeng_high
for lowengphot_tflist in lowengphot_tflist_arr:
    
    lowengphot_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengphot_tflist:
        tf.rebin(photeng)
        # Pad with zeros so that it becomes photeng x photeng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        # Photons in the low energy bins should be immediately deposited.
        tf._grid_vals[0:photeng_low.size, 0:photeng_low.size] = np.identity(photeng_low.size)
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = photeng
        tf._rs = tf.rs[0]*np.ones_like(photeng)
    
    lowengphot_tflist.eng = photeng
    lowengphot_tflist.in_eng = photeng
    
# lowengelec_tflist.in_eng set to photeng_high 
for lowengelec_tflist in lowengelec_tflist_arr:
    
    lowengelec_tflist.at_val('in_eng', photeng_high, bounds_error=False, fill_value=0)
    
    for tf in lowengelec_tflist:
        tf.rebin(eleceng)
        # Pad with zeros so that it becomes photeng x eleceng. 
        tf._grid_vals = np.pad(tf.grid_vals, ((photeng_low.size,0), (0,0)), 'constant')
        tf._N_underflow = np.pad(tf._N_underflow, (photeng_low.size, 0), 'constant')
        tf._eng_underflow = np.pad(tf._eng_underflow, (photeng_low.size, 0), 'constant')
        tf._in_eng = photeng
        tf._eng = eleceng
        tf._rs = tf.rs[0]*np.ones_like(photeng)

    lowengelec_tflist.eng = eleceng
    lowengelec_tflist.in_eng = photeng

## Coarsening

We need to decide on the redshift step to use. The default redshift step in the transfer functions is $d \log (1+z) = 0.002$, with the transfer function with this step size evaluated at (currently) 50 redshifts, from $z = 30$ to $z = 4$. If we stick with this step size, then we must interpolate the transfer function at 2049 points between $z = 30$ and $z = 4$, so that every multiplication by the transfer function advances the spectrum by a step of $d \log (1+z) = 0.002$. 

However, we can coarsen the high energy photon transfer function by some factor $f$ by multiplying the transfer function matrix by itself multiple times, producing transfer functions for a redshift step of $d \log (1+z) = f \times 0.002$. The assumption here is that the transfer function does not change significantly over this bigger redshift step, which appears to be a good approximation in all of the cases checked so far. We will then have to interpolate the transfer function at $2049/f$ steps, and perform $2049/f$ matrix multiplications to evolve the spectrum from $z=30$ to $z=4$. 

This is performed using the function `TransferFuncList.coarsen()`. There are two types of coarsening: one is for a propagation matrix, where we simply multiply the transfer function by itself multiple times (here given by `coarsen_factor`). The other is for a deposition matrix. The deposition matrix is taken to be constant over the number of steps given by `coarsen_factor`, and is applied to the high energy photon spectrum that is produced by each multiplication of the transfer function to get to the next redshift. After coarsening, if the propagation transfer function is $P$, and the deposition transfer function is $D$, coarsening returns $B (1 + P + \cdots + P^{n-1})$. 

After coarsening, the resulting transfer functions are then interpolated at the redshifts given in `rs_list`. 

In [ ]:
# The redshift step to use will be coarsen_factor * 0.002
coarsen_factor = 1
# Generate the redshifts at which we want to get an interpolated value for the transfer function. 
tf_rs_arr = highengphot_tflist_arr[0].rs
rs_list = np.exp(np.arange(np.log(tf_rs_arr[0]), 
                           np.log(tf_rs_arr[-1])-highengphot_tflist_arr[0].dlnz*coarsen_factor, 
                           -highengphot_tflist_arr[0].dlnz*coarsen_factor)
                )

rs_list[-1] = max(4., rs_list[-1])

if coarsen_factor > 1:
    for (
        lowengphot_tflist, lowengelec_tflist, highengphot_tflist
    )in tqdm(zip(
        lowengphot_tflist_arr, lowengelec_tflist_arr, highengphot_tflist_arr
    )):
        lowengphot_tflist.coarsen(coarsen_factor, delete_tfs=False, coarsen_type='dep', prop_transfunclist=highengphot_tflist)
        lowengelec_tflist.coarsen(coarsen_factor, delete_tfs=False, coarsen_type='dep', prop_transfunclist=highengphot_tflist)
        highengphot_tflist.coarsen(coarsen_factor, delete_tfs=False)
    
lowengphot_tflist.at_val('rs',rs_list)
lowengelec_tflist.at_val('rs',rs_list)
highengphot_tflist.at_val('rs',rs_list)

phot_spec_init.rs = rs_list[0]

In [ ]:
rs_list = np.exp(np.arange(np.log(tf_rs_arr[0]), 
                           np.log(tf_rs_arr[-1])-highengphot_tflist_arr[0].dlnz*coarsen_factor, 
                           -highengphot_tflist_arr[0].dlnz*coarsen_factor)
                )
print(rs_list)
print(np.log(rs_list))
print(max(4.,3))

## Transfer Function Checks

Let's do the check for the transfer functions first. We'll just use the first `tflist` in each list as a check.

Now we evolve the initial photon spectrum across all of the redshifts specified in `rs_list`, using the coarsened transfer functions. This is performed by calling the function `spectools.evolve()`, which takes as arguments the initial spectrum, the list of propagation transfer functions, as well as a list of deposition transfer functions. We want to use `evolve_type='prop'` for the high energy photons (just propagation) and `evolve_type='dep'` for low energy electrons and photons, in which case we must specify both propagation and deposition transfer matrices. `save_steps` determines whether the output is a single `Spectrum` at the end of the whole propagation, or a `Spectra` object, which records the spectrum at each intermediate step.

In [ ]:

highengphot_spectra = spectools.evolve(
    phot_spec_init, highengphot_tflist_arr[0], evolve_type='prop', save_steps=True
)
lowengphot_spectra  = spectools.evolve(
    phot_spec_init, lowengphot_tflist_arr[0], evolve_type='dep',
    prop_tflist=highengphot_tflist_arr[0], save_steps=True
)
lowengelec_spectra = spectools.evolve(
    phot_spec_init, lowengelec_tflist_arr[0],
    evolve_type='dep', prop_tflist=highengphot_tflist_arr[0], save_steps=True
)

Here, we process the data from our reference files in order to make the comparison with our result. First, we need to rebin the reference file spectra into the same energy abscissa as our output from above. Then, we switch the spectrum to `'N'` type, so that we can make a direct comparison by plotting `Spectrum.N`. 

In [ ]:
file_highengphot_spectra.rebin(highengphot_spectra.eng)
file_highengphot_spectra.switch_spec_type()
file_lowengphot_spectra.rebin(lowengphot_spectra.eng)
file_lowengphot_spectra.switch_spec_type()
file_lowengelec_spectra.rebin(lowengelec_spectra.eng)
file_lowengelec_spectra.switch_spec_type()

Because we have coarsened our deposition transfer function, the output in `lowengphot_spectra` and `lowengelec_spectra` is actually equivalent to the total deposition from a number of steps given by `coarsen_factor` in the original file. We have to add up all of the deposited low energy electrons and photons in the reference file in order to make the correct comparison.

In [ ]:
combined_file_lowengphot_spectra = Spectra([], spec_type='N')
combined_file_lowengelec_spectra = Spectra([], spec_type='N')

for i in np.arange(np.floor_divide(2049,2*coarsen_factor)):
    mask = np.zeros(2049)
    mask[2*coarsen_factor*i:2*coarsen_factor*i+2*coarsen_factor] = 1
    out_spec_phot = file_lowengphot_spectra.sum_specs(mask)
    out_spec_elec = file_lowengelec_spectra.sum_specs(mask)
    out_spec_phot.rs = file_lowengphot_spectra[2*coarsen_factor*i].rs
    out_spec_elec.rs = file_lowengelec_spectra[2*coarsen_factor*i].rs
    combined_file_lowengphot_spectra.append(out_spec_phot)
    combined_file_lowengelec_spectra.append(out_spec_elec)

In [ ]:
plt.figure()

i_plot = 10

ax = plt.gca()

tf_plot, = highengphot_spectra.plot(ax, ind=rs_list[i_plot], indtype='rs')
highengphot_spectra.plot(ax, ind=rs_list[i_plot+1], indtype='rs')
file_plot, = file_highengphot_spectra.plot(ax, ind=rs_list[i_plot], indtype='rs')
file_highengphot_spectra.plot(ax, ind=rs_list[i_plot+1], indtype='rs')

plt.title('High Energy Photon Spectrum\n' + r'$m_\chi$ ='
          +'{:.2e}'.format(mwimp_arr[eng_ind])
          +r' eV, $1+z$ = '+'{0:.1f}'.format(rs_list[i_plot])
         )

ax.legend((tf_plot, file_plot), ('Transfer Function', 'Reference File'))
ax.set_xscale('log')
ax.set_yscale('log')

plt.xlabel('Secondary Photon Energy [eV]')
plt.ylabel('Number of Photons in Each Bin')
plt.axis([1e4, 2e9, 1e-8, 1e4])

In [ ]:
j = 800
print(highengphot_spectra.rs[j])
print(file_highengphot_spectra.rs[2*j])

In [ ]:
np.set_printoptions(threshold=np.nan)
np.transpose([highengphot_spectra[0].N, highengphot_spectra[1].N, 
              file_highengphot_spectra[2].N, highengphot_spectra[2].N, 
             file_highengphot_spectra[4].N])

In [ ]:
plt.figure()

i_plot = 30

ax = plt.gca()
tf_plot, = lowengphot_spectra.plot(ax, ind=rs_list[i_plot], indtype='rs')
file_plot, = combined_file_lowengphot_spectra.plot(ax, ind=rs_list[i_plot], indtype='rs')

ax.set_xscale('log')
ax.set_yscale('log')

plt.title('Low Energy Photon Spectrum\n' + r'$m_\chi$ ='
          +'{:.2e}'.format(mwimp_arr[eng_ind])
          +r' eV, $1+z$ = '+'{0:.1f}'.format(rs_list[i_plot])
         )

ax.legend((tf_plot, file_plot), ('Transfer Function', 'Reference File'))


plt.xlabel('Secondary Photon Energy [eV]')
plt.ylabel('Number of Photons in Each Bin')
plt.axis([1e-4, 4e12, 1e-20, 1e6])

In [ ]:
plt.figure()

i_plot = 100

ax = plt.gca()
tf_plot, = lowengelec_spectra.plot(ax, ind=rs_list[i_plot], indtype='rs')
file_plot, = combined_file_lowengelec_spectra.plot(ax, ind=rs_list[i_plot], indtype='rs')
# tf_plot, = lowengelec_spectra.plot(ax, ind=rs_list[1], indtype='rs')
# file_plot, = combined_file_lowengelec_spectra.plot(ax, ind=rs_list[1], indtype='rs')

ax.set_xscale('log')
ax.set_yscale('log')
plt.title('Low Energy Electron Spectrum\n' + r'$m_\chi$ ='
          +'{:.2e}'.format(mwimp_arr[eng_ind])
          +r' eV, $1+z$ = '+'{0:.1f}'.format(rs_list[i_plot])
         )

ax.legend((tf_plot, file_plot), ('Transfer Function', 'Reference File'))

plt.xlabel('Secondary Electron Energy [eV]')
plt.ylabel('Number of Electrons in Each Bin')
plt.axis([1e-1, 1e4, 1e-10, 10])

In [ ]:
a = [1,2,3,4,5]
a[0:1]